Les acteurs ont en moyenne quel âge ? données films et acteurs (champs age, birth year et start year)
***age moyen à date du film

1- Appel des deux tables - trie ordre croissant - reset index et drop colonne en trop pour les deux tables

2- Nettoyage de la base Movies :
         - Conservation uniquement des films titleType = movie
         - Retrait des films isadult
         - Suppresion colonne quine me sont plus utiles
         - Explode genre en plusieurs colonnes

3- Nettoyage de la base Actors :
        - Supprimer les acteurs nés avant 1800 pour alléger la base
        - Explode colonne primaryProfession et conserver actor actress
        - Retirer la colonne deathYear
        - Explode colonne knownForTitles en film 1 film 2 etc....
        
        


3- Joindre les tables et retrier :
    - Supprimer les films plus anciens que 1918
    - Suprimer les lignes ou l'âge de l'acteur n'est pas renseigné
(ce trie n'a pas été fait avant car cela permet de nattoyer la base acteurs/movies en même temps)

4 - Créer une colonne âge acteur dans le film : Date du film - date de naissance (startYear - birthYear)

5 - il me faudra une moyenne de cette colonne pour répondre à la question






In [214]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import plotly.express as px 
import plotly.graph_objects as go 
import matplotlib.pyplot as plt
import seaborn as sns

## 1- Appel des deux tables - trie - reset index et drop colonne en trop pour les deux tables

## Table Actors

In [215]:
# appel de la table acteurs

actors = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', sep = '\t', low_memory=False)

In [216]:
# tri des acteurs par date de naissance puis date de décès
actors.sort_values(by=['birthYear','deathYear'], inplace=True)

#reset index 
actors.reset_index(inplace=True)

#drop colonne index en double 
actors.drop('index',axis = 1, inplace=True)



In [217]:
# check si l'ordre est bien modifié et l'index modifié

actors.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0784172,Lucio Anneo Seneca,0004,0065,writer,"tt0049203,tt0972562,tt0218822,tt0237666"
1,nm4511421,Megan Liz Smith,0012,\N,"miscellaneous,actress","tt3416744,tt1966378,tt10183848"
2,nm8810146,Paul Walsh,0021,\N,actor,"tt0098878,tt0314567,tt0093223"
3,nm0430769,Flavius Josephus,0037,0095,"writer,miscellaneous","tt0273236,tt0237666,tt0009573,tt0049833"
4,nm1306202,Plutarch,0046,0122,writer,"tt0237666,tt0272979,tt0056937,tt0472473"


In [218]:
actors.shape

(11366782, 6)

## Table Movies

In [219]:
# appel de la table films (j'ai pris title basics qui contient l'année du film)

movies = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep = '\t', low_memory=False)
movies.sort_values('startYear', inplace=True)

In [220]:
#reset index base movies
movies.reset_index(inplace=True)

#drop colonne index en double 
movies.drop('index',axis = 1, inplace=True)

In [221]:
# check si l'ordre est bien modifié et l'index modifié
movies.head() 

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt3155794,short,Passage de Venus,Passage de Venus,0,1874,\N,1,"Documentary,Short"
1,tt14495706,short,La Rosace Magique,La Rosace Magique,0,1877,\N,1,"Animation,Short"
2,tt2221420,short,Sallie Gardner at a Gallop,Sallie Gardner at a Gallop,0,1878,\N,1,"Documentary,Short"
3,tt12592084,short,Le singe musicien,Le singe musicien,0,1878,\N,1,"Animation,Short"
4,tt7816420,short,Athlete Swinging a Pick,Athlete Swinging a Pick,0,1881,\N,1,"Documentary,Short"


In [222]:
movies.shape # table brute

(8468615, 9)

## 2- Nettoyage de la base Movies :
###          - Conservation uniquement des films titleType = movie
###          - Retrait des films isadult
###          - Suppresion des colonnes qui ne me sont plus utiles
###          - Explode genre en plusieurs colonnes

In [223]:
#Conservation uniquement des films titleType = movie

movies = movies[movies['titleType'] == 'movie']

In [224]:
# Retrait des films isadult - là je décide garder uniquement les 0 qu ne sont pas isAdult
movies = movies[movies['isAdult'] == '0']

In [225]:
# supression des colonnes titleType, endYear, isAdult 

movies = movies.drop(columns=['titleType', 'endYear', 'isAdult'])

In [226]:
#reset index base movies (again !)
movies.reset_index(inplace=True)

#drop colonne index en double 
movies.drop('index',axis = 1, inplace=True)

In [227]:
# table uniquement avec les données conservées
movies.shape 

(584862, 6)

In [228]:
movies.head()

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres
0,tt2210499,Birmingham,Birmingham,1896,61,Documentary
1,tt0229676,Reproduction of the Corbett and Fitzsimmons Fight,Reproduction of the Corbett and Fitzsimmons Fight,1897,\N,"Documentary,News,Sport"
2,tt0236940,69th Regiment Passing in Review,69th Regiment Passing in Review,1898,\N,Documentary
3,tt0235357,Dressing Paper Dolls,Dressing Paper Dolls,1898,\N,Documentary
4,tt0138349,O Carnaval em Lisboa,O Carnaval em Lisboa,1898,\N,Documentary


In [229]:
#Explode genre en plusieurs colonnes
movies[['firstGenre', 'secondyGenre', 'thirdGenre']] = movies['genres'].str.split(',', n=2, expand=True)

#drop colonne genres et thirdGenre
movies.drop(columns=['genres', 'thirdGenre'],axis = 1, inplace=True)


In [230]:
movies.head()

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,firstGenre,secondyGenre
0,tt2210499,Birmingham,Birmingham,1896,61,Documentary,None
1,tt0229676,Reproduction of the Corbett and Fitzsimmons Fight,Reproduction of the Corbett and Fitzsimmons Fight,1897,\N,Documentary,News
2,tt0236940,69th Regiment Passing in Review,69th Regiment Passing in Review,1898,\N,Documentary,None
3,tt0235357,Dressing Paper Dolls,Dressing Paper Dolls,1898,\N,Documentary,None
4,tt0138349,O Carnaval em Lisboa,O Carnaval em Lisboa,1898,\N,Documentary,None


## 3- Nettoyage de la base Actors :
###        - Retirer la colonne deathYear
###        - Remplacer les \N par des nan
###        - Supprimer les acteurs nés avant 1800 pour alléger la base
###        - Explode colonne primaryProfession et conserver actor actress (faire une boucle ou alors remettre les données dans une seule colonne ou les deux : d'abord explode puis réunifier les   colonnes)
###        - Explode colonne knownForTitles en film 1 film 2 etc....

In [231]:
# Retirer la colonne deathYear
actors.drop(columns=['deathYear'], inplace=True)

In [232]:
# contôle colonne drop
actors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11366782 entries, 0 to 11366781
Data columns (total 5 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   nconst             object
 1   primaryName        object
 2   birthYear          object
 3   primaryProfession  object
 4   knownForTitles     object
dtypes: object(5)
memory usage: 433.6+ MB


In [233]:
# remplacer les \N pas des nan dans la colonne birthYear

actors['birthYear'].replace('\\N', np.nan, inplace=True)

In [234]:
# check type colonne birthYear
actors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11366782 entries, 0 to 11366781
Data columns (total 5 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   nconst             object
 1   primaryName        object
 2   birthYear          object
 3   primaryProfession  object
 4   knownForTitles     object
dtypes: object(5)
memory usage: 433.6+ MB


In [235]:
# ne conserve que les rows actors nés avant 1800
actors = actors[actors['birthYear'] >= '1800']

#reset index base actors (again !)
actors.reset_index(inplace=True)

#drop colonne index en double (again !)
actors.drop('index',axis = 1, inplace=True)

In [236]:
actors.head()

,nconst,primaryName,birthYear,primaryProfession,knownForTitles
0,nm3885064,Momin Khan Momin,1800,music_department,tt1645182
1,nm0353596,David Haggart,1801,writer,tt0064988
2,nm0069057,Vincenzo Bellini,1801,"soundtrack,music_department,composer","tt1462900,tt2395427,tt0485947,tt0092563"
3,nm0333187,Christian Dietrich Grabbe,1801,writer,"tt0352909,tt0109781,tt0451590,tt0207421"
4,nm0486937,Joseph Lanner,1801,"soundtrack,composer,music_department","tt0080455,tt0031270,tt0024749,tt0036235"


In [237]:
# remplacer les \N pas des nan dans tout le df actors
actors.replace('\\N', np.nan, inplace=True)

In [238]:
# change type colonne primaryProfession en str
actors['primaryProfession'] = actors['primaryProfession'].astype(str)

In [239]:
actors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536825 entries, 0 to 536824
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   nconst             536825 non-null  object
 1   primaryName        536825 non-null  object
 2   birthYear          536825 non-null  object
 3   primaryProfession  536825 non-null  object
 4   knownForTitles     523837 non-null  object
dtypes: object(5)
memory usage: 20.5+ MB


In [240]:
# explode colonne primaryProfession
actors[['firstProfession', 'secondProfession', 'thirdProfession']] = actors['primaryProfession'].str.split(',', n=2, expand=True)

#drop colonne primaryProfession
actors.drop(columns=['primaryProfession'],axis = 1, inplace=True)

In [251]:
# check dataframe actors
actors.head()

,nconst,primaryName,birthYear,knownForTitles,firstProfession,secondProfession,thirdProfession
0,nm3885064,Momin Khan Momin,1800,tt1645182,music_department,None,None
1,nm0353596,David Haggart,1801,tt0064988,writer,None,None
2,nm0069057,Vincenzo Bellini,1801,"tt1462900,tt2395427,tt0485947,tt0092563",soundtrack,music_department,composer
3,nm0333187,Christian Dietrich Grabbe,1801,"tt0352909,tt0109781,tt0451590,tt0207421",writer,None,None
4,nm0486937,Joseph Lanner,1801,"tt0080455,tt0031270,tt0024749,tt0036235",soundtrack,composer,music_department


In [252]:
# check dataframe before select value actor or actress in three colums : firstProfession, secondProfession, thirdProfession

actors.shape

(536825, 7)

In [256]:
# Je ne garde que les acteurs/actrices

actors = actors[(actors['firstProfession'].isin(['actress','actor'])) | ( actors['secondProfession'].isin(['actress','actor'])) | ( actors['thirdProfession'].isin(['actress','actor']))]

In [258]:
# check dataframe after select value actor or actress in three colums : firstProfession, secondProfession, thirdProfession
actors.shape

(324744, 7)

In [ ]:
#drop colums firstProfession, secondProfession, thirdProfession qui ne me sont plus utiles

actors.drop(columns=['primaryProfession'],axis = 1, inplace=True)

In [259]:
actors.head()

,nconst,primaryName,birthYear,knownForTitles,firstProfession,secondProfession,thirdProfession
446,nm6368729,William H. Taylor,1827,tt0021511,actor,None,None
466,nm0853014,Pop Taylor,1828,"tt0015864,tt0009886",actor,None,None
483,nm0420198,Joseph Jefferson,1829,"tt0000464,tt0004537,tt0006030,tt0012627",actor,writer,None
508,nm0592974,Frédéric Mistral,1830,"tt0161816,tt0492286,tt2813560,tt0764830",writer,actor,None
513,nm0542567,Charles Manley,1830,"tt0004313,tt0000414,tt0000340,tt0000341",actor,None,None


## 3- Joindre les tables et retrier :

In [ ]:
# suppression des lignes Nan rapport à la date de naissance

#actors.dropna(subset=['birthYear'], inplace=True)

In [ ]:
# actors['birthYear']= actors['birthYear'].astype(int)

In [ ]:
# prendre que la colonne birth
#actors_birth = actors['birthYear'] 
#print(actors_birth)

In [ ]:
'''actors_birth2 = actors_birth(['actors_birth'] >= 1850)
print(actors_birth2)

#actors['birthYear'] = datetime.strptime(actors['birthYear'], '%Y')'''

"actors_birth2 = actors_birth(['actors_birth'] >= 1850)\nprint(actors_birth2)\n\n#actors['birthYear'] = datetime.strptime(actors['birthYear'], '%Y')"